# Create island defect on a slab

Create an island defect on a slab

<h2 style="color:green">Usage</h2>

1. Make sure to select Input Materials (in the outer runtime) before running the notebook.
1. Set notebook parameters in cell 1.1. below (or use the default values).
1. Click “Run” > “Run All” to run all cells. 
1. Scroll down to view results. 


## Notes

1. For more information, see [Introduction](Introduction.ipynb)
<!-- # TODO: use a hashtag-based anchor link to interface creation documention above -->


## 1. Prepare the Environment
### 1.1. Set up defect parameters 

In [ ]:
# Choose the island shape: 'cylinder', 'sphere', 'box', or 'triangular_prism'
ISLAND_SHAPE = 'cylinder'

# Common parameters
CENTER_POSITION = [0.5, 0.5, 0.5]  # Center of the island

# Shape-specific parameters
CYLINDER_PARAMS = {
    'radius': 0.25,
    'min_z': 0,
    'max_z': 1
}

SPHERE_PARAMS = {
    'radius': 0.25
}

BOX_PARAMS = {
    'min_coordinate': [0.25, 0.25, 0],
    'max_coordinate': [0.75, 0.75, 1]
}

TRIANGULAR_PRISM_PARAMS = {
    'position_on_surface_1': [0.25, 0.25],
    'position_on_surface_2': [0.75, 0.25],
    'position_on_surface_3': [0.5, 0.75],
    'min_z': 0,
    'max_z': 1
}

### 1.2. Install Packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install` (see [README](../../README.ipynb)).

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip
    await micropip.install('mat3ra-api-examples', deps=False)
    from utils.jupyterlite import install_packages
    await install_packages("", "../../config.yml")

### 1.3. Get input material
Materials are loaded with `get_materials()`.

In [ ]:
from utils.jupyterlite import get_materials
materials = get_materials(globals())

### 1.4. Create and preview Slab

In [ ]:

from mat3ra.made.tools.utils.coordinate import (
    CylinderCoordinateCondition,
    SphereCoordinateCondition,
    BoxCoordinateCondition,
    TriangularPrismCoordinateCondition
)
from mat3ra.made.tools.build.defect import IslandSlabDefectConfiguration

def get_coordinate_condition(shape):
    if shape == 'cylinder':
        return CylinderCoordinateCondition(
            center_position=CENTER_POSITION[:2],
            radius=CYLINDER_PARAMS['radius'],
            min_z=CYLINDER_PARAMS['min_z'],
            max_z=CYLINDER_PARAMS['max_z']
        )
    elif shape == 'sphere':
        return SphereCoordinateCondition(
            center_position=CENTER_POSITION,
            radius=SPHERE_PARAMS['radius']
        )
    elif shape == 'box':
        return BoxCoordinateCondition(
            min_coordinate=BOX_PARAMS['min_coordinate'],
            max_coordinate=BOX_PARAMS['max_coordinate']
        )
    elif shape == 'triangular_prism':
        return TriangularPrismCoordinateCondition(
            position_on_surface_1=TRIANGULAR_PRISM_PARAMS['position_on_surface_1'],
            position_on_surface_2=TRIANGULAR_PRISM_PARAMS['position_on_surface_2'],
            position_on_surface_3=TRIANGULAR_PRISM_PARAMS['position_on_surface_3'],
            min_z=TRIANGULAR_PRISM_PARAMS['min_z'],
            max_z=TRIANGULAR_PRISM_PARAMS['max_z']
        )
    else:
        raise ValueError(f"Unsupported island shape: {shape}")

coordinate_condition = get_coordinate_condition(ISLAND_SHAPE)
slab = materials[0]
if slab.metadata["build"]["type"] != IslandSlabDefectConfiguration.__class__.__name__():
    raise ValueError("The material is not a slab")
island_config = IslandSlabDefectConfiguration(
    crystal=slab,
    condition=coordinate_condition
)

## 2. Create the Defect
### 2.1. Set slab builder parameters

In [ ]:
from mat3ra.made.tools.build.defect import IslandSlabDefectBuilder, SlabDefectBuilderParameters
params = SlabDefectBuilderParameters(
    auto_add_vacuum=True,
    vacuum_thickness=10.0,
)
builder = IslandSlabDefectBuilder(build_parameters=params)

### 2.2. Create the island

In [ ]:
slab_with_island = builder.get_material(island_config)

## 3. Visualize the Slabs with Adatom

In [ ]:
from utils.visualize import visualize_materials as visualize

visualize([{"material": slab, "title": "Original material"},
           {"material": slab_with_island, "title": f"Material with Island Defect ({ISLAND_SHAPE})"}],
rotation="-90x")

visualize([{"material": slab, "title": "Original material"},
           {"material": slab_with_island, "title": f"Material with Island Defect ({ISLAND_SHAPE})"}])

## 4. Pass data to the outside runtime

In [ ]:
from utils.jupyterlite import set_materials

set_materials(slab_with_island)